# Pokemon Type Predictor

## MLP - Tensor Flow

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
#abre e le o arquivo
with open('Pokemon.csv') as file:
    lines = file.readlines()

In [3]:
#separa nome_coluna e data
col_names = lines[0]
raw_data = lines[1:]

In [4]:
#armazena os dados necessarios
data = []
type1_labels = []
type2_labels = []
dict_types = {}
typeID = 0

for line in raw_data:
    separated_data = line.split(',')
    type1 = separated_data[2]
    type2 = separated_data[3]
    
    if type1 not in dict_types:
        dict_types[type1] = typeID
        typeID = typeID + 1
    
    if type2 not in dict_types:
        dict_types[type2] = typeID
        typeID = typeID + 1
        
    temp_stats = []
    for i in separated_data[4:11]:
        temp_stats.append(i)
        
    data.append(temp_stats)
    type1_labels.append(dict_types[type1])
    type2_labels.append(dict_types[type2])

In [5]:
#Separa o dataset em treino e teste
train_data, test_data, train_label, test_label = train_test_split(data, type_labels, test_size = 0.32, random_state=0)

NameError: name 'type_labels' is not defined

In [ ]:
#Converte os dados para o numpy e cria o onehot array para as labels
train_data_np = np.matrix(train_data).astype(np.float32)
train_label_np = np.array(train_label).astype(dtype=np.uint8)

test_data_np = np.matrix(test_data).astype(np.float32)
test_label_np = np.array(test_label).astype(dtype=np.uint8)

train_labels_onehot = (np.arange(typeID) == train_label_np[:, None]).astype(np.float32)
test_labels_onehot = (np.arange(typeID) == test_label_np[:, None]).astype(np.float32)

In [ ]:
train_size,num_features = train_data_np.shape

In [ ]:
X = tf.placeholder("float", shape=[None, num_features])
Y = tf.placeholder("float", shape=[None, typeID])

In [ ]:
# For the test data, hold the entire dataset in one constant node.
test_data_node = tf.constant(test_data_np)

# Define and initialize the network.

# These are the weights that inform how much each feature contributes to
# the classification.
W = tf.Variable(tf.zeros([num_features,typeID]))
b = tf.Variable(tf.zeros([typeID]))
y = tf.nn.softmax(tf.matmul(X,W) + b)

# Optimization.
cross_entropy = -tf.reduce_sum(Y*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# Evaluation.
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [ ]:
#Generate our Tensorflow session
with tf.Session() as s:    
    
    # Run all the initializers to prepare the trainable parameters.
    tf.initialize_all_variables().run()
    
    # Iterate and train.
    for step in range(typeID * train_size // 100):
        offset = (step * 100) % train_size
        print(offset)
        batch_data = train_data_np[offset:(offset + 100), :]
        batch_labels = train_label_np[offset:(offset + 100)]
        costReturn = train_step.run(feed_dict={X: batch_data, Y: batch_labels})
        
    #print ('Weight matrix.')
    #print s.run(W)
    #print
    #print ('Bias vector.')
    #print s.run(b)
    #print
    #print "Applying model to first test instance."
    #first = test_data_np[:1]
    #print "Point =", first
    #print "Wx+b = ", s.run(tf.matmul(first,W)+b)
    #print "softmax(Wx+b) = ", s.run(tf.nn.softmax(tf.matmul(first,W)+b))
    #print
            
    #print "Accuracy:", accuracy.eval(feed_dict={X: test_data, Y: test_labels})


## ---------------------------------------------------------------------------------------

## MLP - Scikit Learn

In [148]:
df = pd.read_csv('Pokemon.csv')
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [ ]:
df.info()

In [ ]:
df[df['Type 1'] == 'Fire'][['HP', 'Attack', 'Defense']].mean()

In [ ]:
df.describe()

In [ ]:
#df[df['Type 1']=='Fire'].plot(kind='scatter', x='Attack', y='Defense', color='red')
#df[df['Type 1']=='Water'].plot(kind='scatter', x='Attack', y='Defense', color='blue')
plt.scatter(df[df['Type 1']=='Water']['Attack'], df[df['Type 1']=='Water']['Defense'],color='blue')
plt.scatter(df[df['Type 1']=='Fire']['Attack'], df[df['Type 1']=='Fire']['Defense'],color='red')
plt.show()

In [ ]:
#correlation map
f,ax = plt.subplots(figsize=(10, 10))
sns.heatmap(df.corr(), annot=True, ax=ax)
plt.show()

In [160]:
df = pd.get_dummies(df, columns=['Type 2'], prefix='type2')

In [161]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Y = df['Type 1']
X = df.drop(['#', 'Name', 'Type 1'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.4, random_state=109)

#cols = X.columns
#cols_to_std = cols.drop(['Type 2'])

scaler = StandardScaler()
X_train_scaled = scaler.fit(X_train).transform(X_train)
X_test_scaled = scaler.transform(X_test)

mlp = MLPClassifier(max_iter=1500, activation='logistic', random_state=42, hidden_layer_sizes=(100, 50))
mlp.fit(X_train_scaled, y_train)

print('Accuracy on the training subset: {:.3f}'.format(mlp.score(X_train_scaled, y_train)))
print('Accuracy on the test subset: {:.3f}'.format(mlp.score(X_test_scaled, y_test)))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

predictions = mlp.predict(X_test_scaled)

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

## -------------------------------------------------------------

## SVM - Scikit Learn

In [163]:
# Split data

X = X[]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=109)

ValueError: Found input variables with inconsistent numbers of samples: [0, 800]

In [8]:
from sklearn import svm
from sklearn import metrics

clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

NameError: name 'X_train' is not defined

## --------------------------------------------------------------

## Using PokeAPI

In [157]:
df = pd.read_csv('Pokemon.csv')
df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [158]:
import requests

#Adding habitat column from PokeAPI
df['Habitat'] = np.nan
df.set_index('Name', inplace = True, drop=False)

for i in range(1, 10):
    resp = requests.get('https://pokeapi.co/api/v2/pokemon-habitat/' + str(i))
    d = resp.json()
    l = []
    for x in range(len(d['pokemon_species'])):
        l.append(d['pokemon_species'][x]['name'])
        l[x] = l[x].capitalize()

    df['Habitat'].loc[df.index.intersection(l)] = i
    
